In [1]:
import serial
import numpy as np
import h5py
import time
import math
import requests
import sys
import os

In [2]:
#Starting with Influx 2.0, authorization tokens are necessary to access databases

#Stringing together query access
INFLUX_TOKEN='AjsrNgY_k97FMvgfCsgc2tPTx-lOVM-aYaCMjymNVIWpoSCkYh7H4AqIV9pLQHHk07zJa5pxTn4lo-3Ashwu5Q=='
ORG="tim@upenn"
INFLUX_CLOUD_URL='localhost'
BUCKET_NAME='sensors'

QUERY_URI='http://{}:8086/api/v2/write?org={}&bucket={}&precision=ns'.format(INFLUX_CLOUD_URL,ORG,BUCKET_NAME)

headers = {}
headers['Authorization'] = 'Token {}'.format(INFLUX_TOKEN)

#Line protocol
location_tag = 'lab'
measurement_name = 'heading'

In [3]:
mag_directory = time.strftime("%Y-%m-%d-Mag")
inclin_directory = time.strftime("%Y-%m-%d-Inclin")
mag_file = h5py.File('/home/user/tim-daq/data/'+mag_directory+'/Mag-20210614-193554.hdf5', 'r', libver = 'latest', swmr = True)
inclin_file = h5py.File('/home/user/tim-daq/data/'+inclin_directory+'/Inclin-20210614-193353.hdf5', 'r', libver = 'latest', swmr = True)

mag_x = mag_file['x']
mag_y = mag_file['y']
mag_z = mag_file['z']
mag_timestamp = mag_file['time']
inclin_x = inclin_file['x']
inclin_y = inclin_file['y']

while True:
    time.sleep(0.13)
    mag_index_num = mag_file['index'][0] 
    inclin_index_num = inclin_file['index'][0]
    #if mag_index_num <
    mag_x.id.refresh()
    mag_y.id.refresh()
    mag_z.id.refresh()
    mag_timestamp.refresh()
    
    inclin_x.id.refresh()
    inclin_y.id.refresh()
    
    mag_x_list = mag_x[:]
    mag_y_list = mag_y[:]
    mag_z_list = mag_z[:]
    mag_timestamp_list = mag_timestamp[:]
    inclin_x_list = inclin_x[:]
    inclin_y_list = inclin_y[:]
    
    mag_x_dat = mag_x_list[mag_index_num]
    mag_y_dat = mag_y_list[mag_index_num]
    mag_z_dat = mag_z_list[mag_index_num]
    mag_timestamp_dat = mag_timestamp_list[mag_index_num]
    
    inclin_x_dat = inclin_x_list[inclin_index_num]
    inclin_y_dat = inclin_y_list[inclin_index_num]
    
    if inclin_y_dat > 0:
        pitch = np.deg2rad(90 - inclin_y_dat)
    else:
        pitch = np.deg2rad(90 + inclin_y_dat)
        
    if inclin_x_dat > 0:
        roll = np.deg2rad(90 - inclin_x_dat)
    else:
        roll = np.deg2rad(90 + inclin_x_dat)
        
    Xh = mag_x_dat * np.cos(pitch) + mag_y_dat * np.sin(roll) * np.sin(pitch) - mag_z_dat * np.cos(roll) * np.sin(pitch)
    
    Yh = mag_y_dat * np.cos(roll) + mag_z_dat * np.sin(roll)
    
    #print(pitch, roll)
    heading = np.rad2deg(math.atan2(mag_x_dat,mag_y_dat))
    #heading = np.rad2deg(math.atan2(Yh,Xh))
    line = '{measurement},location={location} heading={heading} {timestamp}'.format(measurement=measurement_name,location=location_tag, heading=heading, timestamp=mag_timestamp_dat)
    r = requests.post(QUERY_URI, data=line, headers=headers)
    #print(r.status_code)
    #print(line)
    
    #print(index_num)
    #print(x_data[np.max(np.flatnonzero(x_data))])
    #print(mag_x_dat,mag_y_dat,mag_z_dat,"  ",inclin_x_dat,inclin_y_dat)
    
 

KeyboardInterrupt: 

In [4]:
mag_file.close()
inflin_file.close()

NameError: name 'inflin_file' is not defined